In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster = "kenneth-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster)
    print('Existing Cluster found')
except Exception as e:
    print(e + "No existing cluster found. Creating a new cluster.")
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4, vm_priority='lowpriority',
                                                           idle_seconds_before_scaledown=600)
    compute_target = ComputeTarget.create(ws, cluster, config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os


ps = RandomParameterSampling(parameter_space={'C': uniform(0.1, 100)})

policy = BanditPolicy(slack_factor=0.1)

est = SKLearn(source_directory='./train', entry_script='train.py', compute_target=compute_target,
                script_params={ '--C' : 1, '--max_iter': 100})


hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, 
                                     max_duration_minutes=5)
                            
# if "training" not in os.listdir():
#     os.mkdir("./training")

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.core.experiment import Experiment

experiment = Experiment(ws, "kenneth-experiment")
run = experiment.submit(config=hyperdrive_config, show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = run.get_best_run_by_primary_metric()
best_run.download_files(prefix='output')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"


data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(data_url)


In [1]:
import sys
sys.path.append('train')
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

ModuleNotFoundError: No module named 'azureml'

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)

In [ ]:
import numpy as np
import pandas as pd
tr = np.concatenate((x_train, y_train[:, np.newaxis]), axis=-1)
tr_df = pd.DataFrame(data=tr, columns=x.columns.to_list() + ['y'])
tr_df.to_csv('cleaned_data.csv', index=False)

from azureml.core import Workspace, Dataset
#
#
#
#
subscription_id = ''
resource_group = 'Udacity'
workspace_name = 'Udacity_operationalizing'

workspace = Workspace(subscription_id, resource_group, workspace_name)

training_data = Dataset.get_by_name(workspace, name='cleaned_data.csv')

In [ ]:
import azureml.train.automl.utilities
azureml.train.automl.utilities.get_primary_metrics('classification')

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=training_data,
    label_column_name='y',
    n_cross_validations=3
    )


In [2]:
# Submit your automl run
# Submit automl run
experiment = Experiment(ws, "automl")
run = experiment.submit(config=automl_config, show_output=True)

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

best_run, best_model = run.get_output()
joblib.dump(best_model, 'best_automl_model.joblib')

### YOUR CODE HERE ###